# CBCT融合

该工作流接口接收一个CBCT压缩包与上下颌口扫模型，输出融合后的牙齿与CT分割出的其他结构。用户不再需要自己调用各模块

具体调用接口参考 https://www.chohotech.com/docs/cloud-zh/#/workflow/scan-seg-and-cbct-fusion-1

In [ ]:
# 导入必要的包以及定义函数
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np

def _create_colors():
    # 20 high contrast colors
    colors = [[230, 25, 75,255],[60, 180, 75,255],[255, 225, 25,255],\
            [0, 130, 200,255],[245, 130, 48,255],[145, 30, 180,255],[70, 240, 240,255],\
            [240, 50, 230,255],[210, 245, 60,255],[250, 190, 190,255],[0, 128, 128,255],\
            [230, 190, 255,255],[170, 110, 40,255],[255, 250, 200,255],[128, 0, 0,255],\
            [170, 255, 195,255],[128, 128, 0, 255]]
    #np.random.shuffle(colors)
    # gum color
    colors = [[255,255,255,255]] + colors
    return colors

def colored_mesh(mesh, cid):
    COLORS = _create_colors()
    mcopy = mesh.copy()
    mcopy.visual.face_colors = COLORS[cid % 18]
    return mcopy

# 定义调用规则

请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
# 定义调用JSON的基础部分 - 调用的服务名称以及您的用户名信息
json_call = {
  "spec_group": "cbct", # 调用的工作流组， 随API文档发送
  "spec_name": "scan-seg-and-cbct-fusion", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id
}

### 定义回调地址

回调会在工作流运行结束后向指定地址发送一个POST请求，请求内容为一个JSON，内含4个字段

1. workflow_id (str): 对应的workflow_id
2. metadata (dict): 您启动工作流时传入的metadata
3. success (bool): 工作流是否成功（true为成功）
4. reason (str or null): 如果success为true, 该项为null. 否则为失败原因的字符串表示

如果需要回调，请uncomment以下代码块

In [ ]:
# json_call['metadata'] = { #（可选）这里可以加入metadata, 回调时会附带该信息，字典每项限制128个字符
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ #（可选）这里可以加入回调地址
#     {"url": "https://www.baidu.com"} # 这里可以加入多个回调地址，每个都是{"url": "xxx"}格式
# ]

## 定义输入

输入文件必须是一个zip包，包含所有该序列CBCT文件。算法会自动寻找该zip下包含的第一个CBCT序列，如果zip包里的**第一级目录**嵌套了压缩包（zip / rar / tar），算法至多再解压一级

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

json_call["input_data"] = { 
    "upper_jaw_scan": {"type":"drc", "data": upload_file('upper_jaw_scan.drc')},
    "lower_jaw_scan": {"type":"ply", "data": upload_file('lower_jaw_scan.ply')},
    "raw_ct_file": upload_file("cbct.zip")
}

json_call["output_config"]= {
  "teeth": {"type": "ply"},
  "root": {"type": "ply"},
  "fusion": {"type": "ply"},
  "reconstructions": {"type": "ply"},
  "cbct_lower_jaw": {"type": "ply"},
  "cbct_upper_jaw": {"type": "ply"}
}

# 提交请求

使用 `/run` POST方法提交请求

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

# 等待请求完成

使用轮询API等待请求完成，API地址为 `/run/{run_id}`

**强烈推荐使用回调方法**, 参考[定义回调地址](#定义回调地址)，回调方法无论工作流成功与否，在结束时均会向给定地址发送回调信息，您无需轮询结果

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 900: # 最多等15分钟
    time.sleep(3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

# 获取请求结果

使用 `/data/{run_id}` 获得该工作流所有输出数据，使用 `/data/{run_id}/{key}` 获取某一项数据

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

In [ ]:
def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

reconstructions = sum([
    colored_mesh(retrieve_mesh(result["reconstructions"][k]), i) for i,k in enumerate(result["reconstructions"].keys())
], None)

fusion = sum([
    colored_mesh(retrieve_mesh(result["fusion"][k]), i) for i,k in enumerate(result["fusion"].keys())
], None)

cbct_lower_jaw = sum([
    colored_mesh(retrieve_mesh(k), i) for i,k in enumerate(result["cbct_lower_jaw"])
], None)

cbct_upper_jaw = sum([
    colored_mesh(retrieve_mesh(k), i) for i,k in enumerate(result["cbct_upper_jaw"])
], None)

# 可视化

可视化融合结果

In [ ]:
fusion.show()

可视化重建与其他硬组织拼合结果

In [ ]:
reconstructions.show()

可视化实例分割

In [ ]:
(cbct_lower_jaw + cbct_upper_jaw).show()